# NumPy, SciPy and dynamic relaxation
## Introduction
In today's session we will focus on more advanced uses of scientific and computing libraries in Python. The main exercise will allow you to deform a mesh into a relaxed position, given an inverted gravitational force. 

## Aknowledgements
*Dynamic relaxation* exercise was originally written by Shervin Azadi and Pirouz Nourian, 
*NumPy/SciPy introduction* by Kotryna Valeckaite. Last edit on 09/2019

Thanks to Ir. Dirk Rinze Visser for introducing the following reference:
* Adriaenssens, S., P. Block, D. Veenendaal, and C. Williams (2014). Shell Structures for Architecture: Form Finding and Optimization. New York: Routledge. 
* Chapter 2: Review of Dynamic Relaxation with an extension to six degrees of freedom theory by Adriaenssens, S.

------

# NumPy
[Numpy](https://numpy.org/doc/1.17/reference/index.html) ([the user guide](https://numpy.org/doc/1.17/user/index.html#user)) is the core library for scientific computing in Python. It provides a high-performance multi-dimensional array object (i.e. a matrix) and tools for working with these arrays. Namely, it has built-in methods for most of your linear algebra needs.


A numpy array is a grid of values, **all of the same type**, and is indexed by a tuple of nonnegative integers. Let's start with learning how to construct and access them.

In [ ]:
import numpy as np

# Create a rank 1 array (1D matrix)
a = np.array([1, 2, 3])   

print(a)        
print(type(a))            
print(a.shape)            
print(a[0], a[1], a[2]) 
a[0] = 5
print(a) 

### Exercise 1
Perform the pseudocode instructions below

In [ ]:
b = np.array([[1,2,3],[4,5,6]])    

# print out the whole array as it is

# print out the type

# print out the shape of it

# print out the value of 5 by accessing the array

# change the value of the 3 to anything else

# print out the result


Arrays can be made in many other ways and remembering this functionality (or printing out a cheat sheet) will save you precious coding time. Also, you are **not** limited to 2 dimensions: all examples below can be extended by 1 dimension by adding an additional value after the comma.

In [ ]:
# Creates an array of all zeros
a = np.zeros((2,2))  
print(a)              
                      
# Creates an array of all ones
b = np.ones((2,3))    
print('\n',b)              

# Creates an array of all of a specific number (5 in this case)
c = np.full((2,2), 5)  
print('\n',c)               
                       
# Creates an identity matrix of a specified size (3 in this case)
d = np.eye(3)         
print('\n',d)              
                      
# Creates a 2x3 array filled with random values in [0,5)
e = np.random.random((2,5))  
print('\n',e)    

### Exercise 2
Create an identity matrix of size 3, then change all the ones to a different value each.

## Why use it?
The reason for using it lies not only in the convenience of not having to write your own methods. Namely, the underlying code is based on FORTRAN 77 and C libraries. In human words, it is way faster than writing your own code.

### Exercise 3
To compare the performance of pure Python and Numpy, follow the pseudocode instructions in the next cells.

In [187]:
my_list = []
# Populate my_list, such that it contains 10**6 lists of [2,2,2]
???

In [188]:
def python_test(my_list):
    # TODO: multiply each number in my_list by 3

# this is called a magic command, check out: 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
%time python_test(my_list)        

Wall time: 1.49 s


In [189]:
# TODO: create an array of 3x10**6 containing only 2s
array = ???

In [190]:
def numpy_test(array):
    # TODO: multiply each number in the array by 3
   
    
%time numpy_test(array) 

Wall time: 12 ms



# SciPy
SciPy is an open-source Python library used for scientific and technical computing. It has several sub-packages, such as on covering constants, spatial algorithms (e.g. distance, nearest neighbor fuctions), linear algebra methods and many more. It is built on top of NumPy, making it not only a useful, but also a powerful tool. More about it [here](https://www.tutorialspoint.com/scipy/index.htm).

This library will not be used in the following exercise, but it will prove very useful if you need to do more complex linear algebra routines, such as integration.

# Dynamic relaxation
This exercise consists of 3 main steps:
1. Reading in the data and defining the initial state of the mesh
2. Relaxing the mesh
3. Plotting the results


In [193]:
# For step 1
import copy

# For step 2
from numpy import array, zeros, float64, append, sqrt, sign

# For step 3
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# necessary only for the Jupyter notebook
%matplotlib notebook 

## 1. Reading in the data and defining the initial state of the mesh
During this step we need to read in the data, parse it to something we can manipulate, pre-process the geometeries and pass the results/ initial state of the mesh to the next process. The expected outputs of this step are:
* points : List of points
* edges : List of edges
* vertices : List of vertices
* xyzIS : Initial state of the system
* bbmax, bbmin : Bounding box of the mesh

-------

Let's first declare some of these variables:

In [194]:
# initiating the output
xyz = []  # list of vertex xyz coordinates (it is a list of lists)
vertices = []  # unique xyz coordinates
points = []  # references to vertices
lines = []  # pairs of references to vertices

# initiating the bounding box max and min point
bbmax = [0,0,0]
bbmin = [0,0,0]

The only **input** for the first step of the exercise in the location of the *mesh file (.obj)* you received together with this notebook.

To begin with, let's locate the file and open it:

In [198]:
filename = ??? # TODO: fill in the name of the file and potentially the location

# opening the file
with open(filename, 'r') as myfile:
    data = myfile.read()
    # TODO: split the data by lines
    data = ???

### What is inside of data? (for a more elaborate description of the format, check out this [link](http://paulbourke.net/dataformats/obj/))

In [200]:
data

'# Rhino\n\nv 0 0 0\nv 0 1 0\nv 0 2 0\nv 0 3 0\nv 0 4 0\nv 0 5 0\nv 0 6 0\nv 0 7 0\nv 0 8 0\nv 0 9 0\nv 0 10 0\nv 1 0 0\nv 1 1 0\nv 1 2 0\nv 1 3 0\nv 1 4 0\nv 1 5 0\nv 1 6 0\nv 1 7 0\nv 1 8 0\nv 1 9 0\nv 1 10 0\nv 2 0 0\nv 2 1 0\nv 2 2 0\nv 2 3 0\nv 2 4 0\nv 2 5 0\nv 2 6 0\nv 2 7 0\nv 2 8 0\nv 2 9 0\nv 2 10 0\nv 3 0 0\nv 3 1 0\nv 3 2 0\nv 3 3 0\nv 3 4 0\nv 3 5 0\nv 3 6 0\nv 3 7 0\nv 3 8 0\nv 3 9 0\nv 3 10 0\nv 4 0 0\nv 4 1 0\nv 4 2 0\nv 4 3 0\nv 4 4 0\nv 4 5 0\nv 4 6 0\nv 4 7 0\nv 4 8 0\nv 4 9 0\nv 4 10 0\nv 5 0 0\nv 5 1 0\nv 5 2 0\nv 5 3 0\nv 5 4 0\nv 5 5 0\nv 5 6 0\nv 5 7 0\nv 5 8 0\nv 5 9 0\nv 5 10 0\nv 6 0 0\nv 6 1 0\nv 6 2 0\nv 6 3 0\nv 6 4 0\nv 6 5 0\nv 6 6 0\nv 6 7 0\nv 6 8 0\nv 6 9 0\nv 6 10 0\nv 7 0 0\nv 7 1 0\nv 7 2 0\nv 7 3 0\nv 7 4 0\nv 7 5 0\nv 7 6 0\nv 7 7 0\nv 7 8 0\nv 7 9 0\nv 7 10 0\nv 8 0 0\nv 8 1 0\nv 8 2 0\nv 8 3 0\nv 8 4 0\nv 8 5 0\nv 8 6 0\nv 8 7 0\nv 8 8 0\nv 8 9 0\nv 8 10 0\nv 9 0 0\nv 9 1 0\nv 9 2 0\nv 9 3 0\nv 9 4 0\nv 9 5 0\nv 9 6 0\nv 9 7 0\nv 9 8 0\nv 9 9 0

### How do we make this readable?

In [201]:
# parsing the obj file, filling the data lists
for line in data:
    parts = ??? # TODO: split the string line by ' '

    # checking if it contains data, if not skip the line
    if not parts:
        continue
        
    # assigning the 1st element to head and the rest to tail 
    head = ??? # TODO: get the first element in list of parts
    tail = ??? # TODO: get the all other elements in list of parts
    
    # appending vertex cordinations in (xyz) and assigning an index to them in (points)
    if head == 'v':
        # This is a smaller way to write out for loops
        # To understand read: https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions
        ftail = [float(x) for x in tail] 
        
        xyz.append(ftail)
        points.append(len(xyz)-1)
        
    # iterating through the edges of the faces and appending them in (lines)
    elif head == 'f':
        ftail = [float(x) for x in tail]
        for i in range(len(ftail)):
            sp = ftail[i%len(ftail)]
            ep = ftail[(i+1)%len(ftail)]
            lines.append((int(sp)-1, int(ep)-1))


SyntaxError: invalid syntax (<ipython-input-201-cd023edf9a77>, line 11)

### How do we preprocess the data?

In [ ]:
# points to vertices (welding similar vertices)
x2v = {} # this is the dictionary we will use to map the points to vertices
tol = 0.001 ** 2 # this is the tolerance

# for each point
for i, x in enumerate(xyz):
    found = False
    for j, v in enumerate(vertices):
        if (x[0] - v[0]) ** 2 < tol \
                and (x[1] - v[1]) ** 2 < tol \
                and (x[2] - v[2]) ** 2 < tol:
            found = True
            x2v[i] = j
            break # get out of this loop, if a match is found
    if not found:
        x2v[i] = len(vertices)
        vertices.append(x)
    
    # TODO: finding the bounding box
    # for number in range 3
        # if the current coordinate is bigger than the biggest current coordinate, replace the max coordinate
        # if the current coordinate is smaller...


### Anything we forgot?

In [ ]:
# redefining the indexes: remapping the points to new, velded locations
points[:] = [x2v[index] for index in points]

# redefining the edges by the refined indexes
edges = [(x2v[u[0]], x2v[u[1]]) for u in lines]

# reserving the Initial State of the system
xyzIS = copy.deepcopy(vertices)

### How does the initial state look like?

In [ ]:
xyzIS

# Dynamic relaxation
### Input data:
* vertices: a list of unique xyz coordinates
* edges: a list of pairs of vertex indices
* points: index list of fixed vertices
* bbmax, bbmin : Bounding box coordinate list

### Output data:
* xyzFS : Final state of the system

In [ ]:
# fixed points here are set to be the boundary of the rectangle
fixed = []
for i, pnt in enumerate(iter(xyz)):
    # check if the points are no closer to 
    if abs(pnt[0] - bbmin[0]) < 0.01  \
        or abs(pnt[0] - bbmax[0]) < 0.01 \
        or abs(pnt[1] - bbmin[1]) < 0.01 \
        or abs(pnt[1] - bbmax[1]) < 0.01:
        fixed.append(i)

In [ ]:
no_vert = len(vertices)
no_ed = len(edges)

freePoints = list(set(range(no_vert)) - set(fixed))

xyz = array(vertices)

# p : Applied Load Component which in our case is zero
p = # create an NumPy array of zeroes of shape no_vert x 3 of type float64

# W : Gravity Force
W = array([array([0,0,9.8]) for i in range(no_vert)])

# v : Velosity
v = zeros((no_vert, 3), dtype=float64)


# restlength : Rest Length of the springs
diff = array([(xyz[edges[i][1]] - xyz[edges[i][0]]) for i in range(no_ed)])

restlength = array([sqrt(diff[i,0]*diff[i,0] + diff[i,1]*diff[i,1] + diff[i,2]*diff[i,2]) for i in range(no_ed)])

### We need to make some assumptions, while making this code...

In [ ]:
# adamp : Acceleration damp controls the stability of the system
adamp = 10
IterationMax = 100

# K : Elasticity Constant in Hooke's law
K = 20

# dt : Time intervals
interval = 0.1
mass = 1

In [ ]:
# this is a loop, in which we progress through time
for k in range(IterationMax):
    
    # S : Stiffness Force
    S = ??? # TODO: create an NumPy array of zeroes of shape no_vert x 3 of type float64
    # R : Final Forces
    R = ??? # TODO: create an NumPy array of zeroes of shape no_vert x 3 of type float64
        
    # xyz0 : Initial Position
    xyz0 = xyz.copy()
    # vp : Initial Velocity
    vp = v.copy()
    
    # calculating the length of the edges
    di = array([(xyz[edges[i][1]] - xyz[edges[i][0]]) for i in range(no_ed)])
    dist = array([sqrt(di[i,0]*di[i,0] + di[i,1]*di[i,1] + di[i,2]*di[i,2]) for i in range(no_ed)])

    # Hooke's law : Force = length difference * Elasticity Constant (K)
    EdgeForce = ??? # TODO: make the formula above
    # Edge Force : Decomposing the existing force in each edge into the 3 dimensions 
    EdgeForceS = zeros((no_ed, 3), dtype=float64)

    # iterating throughout the edges
    for i in range(no_ed):
        for j in range(3):
            # Separating the forces into X, Y, Z dimensions
            EdgeForceS[i,j] = (di[i,j] / dist[i]) * EdgeForce[i]

        # Adding the force of each edge to the coresponding nodes
        S[edges[i][0]] -= EdgeForceS[i]
        S[edges[i][1]] += EdgeForceS[i]
    
    R[freePoints] = p[freePoints] + W[freePoints] - S[freePoints]
    
    A = 1/(1 + adamp * dt * 0.5)
    B = (1 - adamp * dt * 0.5 ) / (1 + adamp * dt * 0.5)
    
    # updating the velocity
    v[freePoints] = A * dt * R[freePoints] / mass + B * vp[freePoints]
    # updating the position
    xyz[freePoints] = xyz0[freePoints] + dt * v[freePoints]


xyzFS = xyz

# 3. Plotter

The last step of the process uses Matplotlib library used in the previous workshop. In this step we will create a visualization for the initial state and/or final state of the system. Our inputs are:

* xyzIS : initial state of the system
* xyzFS : final state of the system
* *Optional* bbmax, bbmin : Bounding box coordinate list

In [ ]:
coordinates = xyzFS

# initiating the plotter
fig = plt.figure()
ax = fig.gca(projection='3d')

# plotting edges
for u, v in edges:
    ax.plot([coordinates[u][0], coordinates[v][0]],
            [coordinates[u][1], coordinates[v][1]],
            [coordinates[u][2], coordinates[v][2]], color='k')

# plotting points
for i in points:
    ax.scatter(coordinates[i][0], coordinates[i][1], coordinates[i][2],
               color='r', s=4)
    
# This command changes the scale of the axes
ax.scatter([bbmin[0], bbmax[0]], [bbmin[1], bbmax[1]], [bbmin[2], bbmax[2]+10], color='w', s=1)

# These are needed for the 
# ax.set_xlim([bbmin[0], bbmax[0]])
# ax.set_ylim([bbmin[1], bbmax[1]])
# ax.set_zlim([bbmin[2], bbmax[2]+10])

plt.show()

# A library that works a bit faster ...
[Plotly](https://plot.ly/python/) is another open source library for Python, that works significantly faster than Matplotlib on Jupyter notebooks. Also, it's a bit easier to use: see for yourself below.

In [205]:
import plotly.graph_objects as go

# Here we extract the mesh coordinates from the final state NumPy array
xFS, yFS, zFS = xyzFS.T

# Initial state data is still in list format, so we first convert it to a NumPy array
xIS, yIS, zIS = np.array(xyzIS).T

fig = go.Figure(data=[  go.Mesh3d(x=xIS, y=yIS, z=zIS, color='blue', opacity=0.5),
                        go.Mesh3d(x=xFS, y=yFS, z=zFS, colorscale='Viridis', opacity=0.5),
                        go.Scatter3d(mode = 'markers', x=xFS, y=yFS, z=zFS,
                            marker=dict(size=3, color='red'), showlegend=False),
                        go.Scatter3d(mode = 'markers', x=xIS, y=yIS, z=zIS,
                            marker=dict(size=3, color='blue'), showlegend=False)
                     ]
               )
fig.show()

In [206]:
points = go.Scatter3d(
    mode = 'markers',
    showlegend = False,
    x=xFS, y=yFS, z=zFS,
    marker=dict(
        size=4,
        color='red',
       ))

coordinates = xyzFS
lines = []
for u, v in edges:
    lines.append(go.Scatter3d(
#         mode='lines',
        x = [coordinates[u][0], coordinates[v][0]],
        y = [coordinates[u][1], coordinates[v][1]],
        z = [coordinates[u][2], coordinates[v][2]],
        line=dict(color='black'),
        marker=dict(color='red', size=4),
        showlegend = False
                ))
        
fig = go.Figure(data=lines)

fig.show()
